# Import the Libraries

In [21]:
from binance import Client as c
from binance import BinanceSocketManager as bsm
from binance import ThreadedDepthCacheManager as tdcm

import mplfinance as mpf
import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import math
import os
import time
import glob
from datetime import datetime, timedelta
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

# Access API, Extract USDT Crypto pairs

In [2]:
api_key = 'YOUR OWN PUBLIC KEY'
api_secret = 'YOUR OWN API SECRET KEY'

client = c(api_key, api_secret)

In [3]:
#Access all ticket + immedate price
tickers = client.get_all_tickers()
#put them into a DF
tickers_df = pd.DataFrame(tickers)

#Filter out only ticker that is pair USDT
tickers_df = tickers_df[tickers_df['symbol'].str.contains('USDT')]

#Use Symbol as the index instead of default index
tickers_df.set_index('symbol', inplace =True) #inplace=True apply the function to the dataframe

#check the DF
tickers_df.shape

(426, 1)

In [6]:
# use this code to check for a specific pair
tickers_df.loc['BTCUSDT']

price    20619.15000000
Name: BTCUSDT, dtype: object

In [5]:
# Check he DF:
tickers_df.head()

,price
symbol,
BTCUSDT,20619.15000000
ETHUSDT,1558.13000000
BNBUSDT,298.40000000
BCCUSDT,448.70000000
NEOUSDT,8.65000000


In [12]:
# A fast way to Call the latest symbol 
tickers_df.index[tickers_df.shape[0]-1]

'OSMOUSDT'

## Extracting K-line of USDT pairs 1 DAY

In [11]:
#HISTORICAL DATA IS K-line
for i in range(0,len(tickers_df.index)-1):
    
    # Access all the K-line data (all price and volume) of a symbol
    #Access date 2000 days ago: Binance listed the crypto in Aug 2017 so 2000 days ago will cover all crypto listed since 2017
    date_2K_ago = str(datetime.now() + timedelta(days=-2000))
    
    historical = client.get_historical_klines(tickers_df.index[i], client.KLINE_INTERVAL_1DAY, start_str=date_2K_ago)
    
    # Put the data into a DF
    hist_df = pd.DataFrame(historical)

    # Name the columns then convert the Open and Close time to second
    hist_df.columns = ['Open time', 'Open', 'High', 'Low', 'Close','Volume', 'Close time', 'Quote asset volume',
                       'Number of trades','Taker buy base asset volume', 'Taker buy quote asset volume', 'ignore']
    hist_df['Open time'] = pd.to_datetime(hist_df['Open time']/ 1000, unit ='s')
    hist_df['Close time'] = pd.to_datetime(hist_df['Close time']/ 1000, unit ='s')

    # Convert all numeric columns into the right format: float64
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume', 'Taker buy base asset volume', 'Taker buy quote asset volume']
    hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)

    # Add a column with a symbol of the crypto
    hist_df['symbol'] = tickers_df.index[i]
    
    #Save to .csv 
    path = r'C:\Users\ethai\OneDrive\1.1. Education\1. BrainStation\0. Capstone\data_1D\ ' #super important to have 'r' and '\ ' -backward slash with space in the end
    hist_df.to_csv(path + tickers_df.index[i]+'_1D.csv')

## Extracting K-line of USDT pairs 1H

In [16]:
#HISTORICAL DATA IS K-line
for i in range(59,len(tickers_df.index)-1):
    
    # Access all the K-line data (all price and volume) of a symbol
    #Access date 2000 days ago: Binance listed the crypto in Aug 2017 so 2000 days ago will cover all crypto listed since 2017
    date_2K_ago = str(datetime.now() + timedelta(days=-2000))
    
    historical = client.get_historical_klines(tickers_df.index[i], client.KLINE_INTERVAL_1HOUR, start_str=date_2K_ago)
    
    # Put the data into a DF
    hist_df = pd.DataFrame(historical)

    # Name the columns then convert the Open and Close time to second
    hist_df.columns = ['Open time', 'Open', 'High', 'Low', 'Close','Volume', 'Close time', 'Quote asset volume',
                       'Number of trades','Taker buy base asset volume', 'Taker buy quote asset volume', 'ignore']
    hist_df['Open time'] = pd.to_datetime(hist_df['Open time']/ 1000, unit ='s')
    hist_df['Close time'] = pd.to_datetime(hist_df['Close time']/ 1000, unit ='s')

    # Convert all numeric columns into the right format: float64
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume', 'Taker buy base asset volume', 'Taker buy quote asset volume']
    hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)

    # Add a column with a symbol of the crypto
    hist_df['symbol'] = tickers_df.index[i]
    
    #Save to .csv 
    path = r'C:\Users\ethai\OneDrive\1.1. Education\1. BrainStation\0. Capstone\data_1H\ ' #super important to have 'r' and '\ ' -backward slash with space in the end
    hist_df.to_csv(path + tickers_df.index[i] + '_1H.csv')

## Merging .csv files

In [50]:
#FOR 1D 
# Define the file extension
all_files_1D = glob.glob('data_1D/*1D.csv')
all_files_1D
# combine all files in the list
combined_1D = pd.concat((pd.read_csv(f, index_col=None, header=0) for f in all_files_1D))
# export to csv
combined_1D.to_csv("1D_combined.csv")

Use this website to learn about difference type of options other than csv
https://towardsdatascience.com/still-saving-your-data-in-csv-try-these-other-options-9abe8b83db3a

In [53]:
#FOR 1H
# Define the file extension
all_files_1H = glob.glob('data_1H/*1H.csv')
all_files_1H
# combine all files in the list
combined_1H = pd.concat((pd.read_csv(f, index_col=None, header=0) for f in all_files_1H))
# export to pickle to save size as csv cant handle large size
combined_1H.to_pickle("1H_combined.pkl")

## Open .pkl - pickle

In [56]:
combined_1H_P = pd.read_pickle('1H_combined.pkl')
combined_1H_P

,Unnamed: 0,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,ignore,symbol
0,0,2021-04-15 07:00:00,10.0000,10.2500,9.9700,10.0700,9367.65,2021-04-15 07:59:59.999000064,95067.4560,816,7204.75,73150.5649,0,1INCHDOWNUSDT
1,1,2021-04-15 08:00:00,10.0300,10.3000,9.7400,10.2100,2426.47,2021-04-15 08:59:59.999000064,24341.4337,197,1115.29,11307.3662,0,1INCHDOWNUSDT
2,2,2021-04-15 09:00:00,10.1700,10.2500,9.8600,9.9200,2581.83,2021-04-15 09:59:59.999000064,25920.0928,200,1799.05,18031.0808,0,1INCHDOWNUSDT
3,3,2021-04-15 10:00:00,9.9500,10.2000,9.7500,9.9700,1918.39,2021-04-15 10:59:59.999000064,19040.8830,224,970.42,9642.8277,0,1INCHDOWNUSDT
4,4,2021-04-15 11:00:00,9.9900,9.9900,8.7800,9.1000,9166.34,2021-04-15 11:59:59.999000064,84599.2323,727,7616.09,70373.0222,0,1INCHDOWNUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32069,32069,2022-10-28 20:00:00,0.2579,0.2599,0.2579,0.2579,125605.00,2022-10-28 20:59:59.999000064,32538.2239,246,79718.00,20655.4531,0,ZRXUSDT
32070,32070,2022-10-28 21:00:00,0.2583,0.2586,0.2577,0.2579,129227.00,2022-10-28 21:59:59.999000064,33370.5802,138,65807.00,16998.5108,0,ZRXUSDT
32071,32071,2022-10-28 22:00:00,0.2582,0.2587,0.2576,0.2587,49585.00,2022-10-28 22:59:59.999000064,12801.0488,112,34250.00,8845.4211,0,ZRXUSDT
32072,32072,2022-10-28 23:00:00,0.2589,0.2591,0.2576,0.2579,43854.00,2022-10-28 23:59:59.999000064,11337.1695,82,18333.00,4739.0808,0,ZRXUSDT


In [64]:
combined_1H_P.drop(["Unnamed: 0", 'ignore'], axis=1)

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,symbol
0,2021-04-15 07:00:00,10.0000,10.2500,9.9700,10.0700,9367.65,2021-04-15 07:59:59.999000064,95067.4560,816,7204.75,73150.5649,1INCHDOWNUSDT
1,2021-04-15 08:00:00,10.0300,10.3000,9.7400,10.2100,2426.47,2021-04-15 08:59:59.999000064,24341.4337,197,1115.29,11307.3662,1INCHDOWNUSDT
2,2021-04-15 09:00:00,10.1700,10.2500,9.8600,9.9200,2581.83,2021-04-15 09:59:59.999000064,25920.0928,200,1799.05,18031.0808,1INCHDOWNUSDT
3,2021-04-15 10:00:00,9.9500,10.2000,9.7500,9.9700,1918.39,2021-04-15 10:59:59.999000064,19040.8830,224,970.42,9642.8277,1INCHDOWNUSDT
4,2021-04-15 11:00:00,9.9900,9.9900,8.7800,9.1000,9166.34,2021-04-15 11:59:59.999000064,84599.2323,727,7616.09,70373.0222,1INCHDOWNUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...
32069,2022-10-28 20:00:00,0.2579,0.2599,0.2579,0.2579,125605.00,2022-10-28 20:59:59.999000064,32538.2239,246,79718.00,20655.4531,ZRXUSDT
32070,2022-10-28 21:00:00,0.2583,0.2586,0.2577,0.2579,129227.00,2022-10-28 21:59:59.999000064,33370.5802,138,65807.00,16998.5108,ZRXUSDT
32071,2022-10-28 22:00:00,0.2582,0.2587,0.2576,0.2587,49585.00,2022-10-28 22:59:59.999000064,12801.0488,112,34250.00,8845.4211,ZRXUSDT
32072,2022-10-28 23:00:00,0.2589,0.2591,0.2576,0.2579,43854.00,2022-10-28 23:59:59.999000064,11337.1695,82,18333.00,4739.0808,ZRXUSDT
